
# 🧭 C5.1.2 — Stratégie de résolution (v2, sans doublons)

**But :** détailler la **stratégie** (tâches, algos, plan d'expérience) **sans répéter** l'EDA/normalisation de C5.1.1.  
On **référence** les KPI définis en 1.1 et on **lit** le dataset *normalisé* produit par 1.1.



## 🎙️ Discours (1–2 min)
> « Nous décomposons en **trois sous-tâches** : Émotions (multi-label), **ABSA** (aspects + polarité), **Sarcasme**.  
> Démarche **progressive** : baselines `scikit-learn` → extensions `HF` si ROI.  
> Nous **réutilisons** le dataset **normalisé** de C5.1.1 pour garantir la **reproductibilité** et éviter les doublons.  
> Les **KPI** fixés en 1.1 guident les choix (F1-macro, AUC, latence). »


## ⚙️ 0) Configuration

In [1]:

from pathlib import Path

# On consomme le dataset normalisé émis par C5.1.1
PARQUET_NORM = Path("data/interim/amazon_electronics_normalized.parquet")
CSV_RAW      = Path("C:/Users/antoi/OneDrive/Documents/Ynov/Projet fil rouge/Bloc 5/amazon-reviews-insights/data/amazon_insights.csv")  # fallback si besoin

PARQUET_NORM, CSV_RAW

(WindowsPath('data/interim/amazon_electronics_normalized.parquet'),
 WindowsPath('C:/Users/antoi/OneDrive/Documents/Ynov/Projet fil rouge/Bloc 5/amazon-reviews-insights/data/amazon_insights.csv'))

## 📥 1) Chargement (sans re-normaliser)

In [2]:

import pandas as pd

if PARQUET_NORM.exists():
    df = pd.read_parquet(PARQUET_NORM)
    print("✓ Chargé depuis le dataset normalisé :", PARQUET_NORM)
else:
    raise FileNotFoundError("Le dataset normalisé est introuvable. Exécute d'abord C5.1.1 pour le générer.")
    
# Vérif colonnes minimales attendues
need = ["review_body","star_rating"]
assert all(c in df.columns for c in need), f"Colonnes manquantes {need} dans le dataset normalisé."
print("Shape:", df.shape)
df.head(3)

✓ Chargé depuis le dataset normalisé : data\interim\amazon_electronics_normalized.parquet
Shape: (1314720, 11)


,review_body,review_title,star_rating,review_date,product_title,product_id,helpful_vote,verified_purchase,main_category,categories,user_id
0,All OK and as described.,Five Stars,5.0,2015-02-04 20:13:26,OOOUSE XT60 Connector Pairs - Pack of 5 pairs,B005FAPYXS,0,True,Industrial & Scientific,['Electronics' 'Home Audio' 'Home Audio Access...,AES2QJS66UB66K2C7MNRDSEOOGAQ
1,Great but how can you go wrong.,Five Stars,5.0,2016-08-21 00:48:38,OOOUSE XT60 Connector Pairs - Pack of 5 pairs,B005FAPYXS,0,True,Industrial & Scientific,['Electronics' 'Home Audio' 'Home Audio Access...,AE5W7TDLLOZD62AGZRS56SOM7KOA
2,Don't waste your money!! Connectors won't plu...,Very poor manufacturing,1.0,2017-03-06 15:53:57,OOOUSE XT60 Connector Pairs - Pack of 5 pairs,B005FAPYXS,1,True,Industrial & Scientific,['Electronics' 'Home Audio' 'Home Audio Access...,AF76HWTTIDICDKSGWLHROHVXO4HA


## 🧩 2) Décomposition des tâches & algorithmes

In [3]:

tasks = pd.DataFrame([
    {"Tâche": "Émotions (multi-label)", "Baseline": "TF-IDF + LogReg (proxy)", "Extension": "GoEmotions (HF)"},
    {"Tâche": "ABSA (aspects + polarité)", "Baseline": "KeyBERT/spaCy + LogReg", "Extension": "HF (token/seq labeling)"},
    {"Tâche": "Sarcasme/ironie", "Baseline": "Proxy/erreurs + modèle simple", "Extension": "roberta-base-irony + calibration"}
])
tasks


,Tâche,Baseline,Extension
0,Émotions (multi-label),TF-IDF + LogReg (proxy),GoEmotions (HF)
1,ABSA (aspects + polarité),KeyBERT/spaCy + LogReg,HF (token/seq labeling)
2,Sarcasme/ironie,Proxy/erreurs + modèle simple,roberta-base-irony + calibration


## 🧱 3) Recettes Features/Modèles (prêtes pour C5.2.x)

In [4]:

from dataclasses import dataclass

@dataclass
class FeatureRecipe:
    kind: str               # 'tfidf_word' | 'tfidf_char' | 'sbert' (plus tard)
    selectk: int = 20000
    svd: int = 0

@dataclass
class ModelRecipe:
    name: str               # 'logreg' | 'complement_nb' | 'hgb_svd'
    calibration: str = 'isotonic'

MVP_FEATURES = FeatureRecipe(kind='tfidf_word', selectk=20000, svd=0)
MVP_MODEL   = ModelRecipe(name='logreg', calibration='isotonic')
MVP_FEATURES, MVP_MODEL

(FeatureRecipe(kind='tfidf_word', selectk=20000, svd=0),
 ModelRecipe(name='logreg', calibration='isotonic'))

## 🎯 4) KPI (rappel rapide depuis C5.1.1)

In [5]:
kpi = pd.DataFrame([
    {"KPI": "Émotions F1-macro", "Cible": 0.60},
    {"KPI": "ABSA F1-macro", "Cible": 0.60},
    {"KPI": "Sarcasme AUC", "Cible": 0.75},
    {"KPI": "Latence démo (ms)", "Cible": 200},
])
kpi

,KPI,Cible
0,Émotions F1-macro,0.60
1,ABSA F1-macro,0.60
2,Sarcasme AUC,0.75
3,Latence démo (ms),200.00


## 🧭 5) Plan d'expérience (baseline → extensions)

In [6]:

plan = [
    "1) Baseline binaire (TF-IDF word + LogReg, SelectK, calibration isotonic)",
    "2) TF-IDF char + ComplementNB en comparaison",
    "3) Option SVD + HGB si gain sur AP/ROC",
    "4) Émotions multi-label (HF) si besoin de granularité",
    "5) ABSA : aspects (KeyBERT/spaCy) + polarité",
    "6) Sarcasme : modèle HF + calibration",
    "7) Validation : PR-AUC, F1@seuil*, matrice confusion, error analysis",
    "8) Sauvegarde artefact + meta + monitoring (drift)"
]
for s in plan:
    print("•", s)


• 1) Baseline binaire (TF-IDF word + LogReg, SelectK, calibration isotonic)
• 2) TF-IDF char + ComplementNB en comparaison
• 3) Option SVD + HGB si gain sur AP/ROC
• 4) Émotions multi-label (HF) si besoin de granularité
• 5) ABSA : aspects (KeyBERT/spaCy) + polarité
• 6) Sarcasme : modèle HF + calibration
• 7) Validation : PR-AUC, F1@seuil*, matrice confusion, error analysis
• 8) Sauvegarde artefact + meta + monitoring (drift)


## 📒 6) Registre d’expériences (structure à remplir en C5.2.x)

In [7]:

import pandas as pd, time
exp_cols = ["run_id","features","model","selectk","svd","calibration","AP","ROC_AUC","F1@0.5","F1@t*","threshold*","notes","created"]
experiments = pd.DataFrame(columns=exp_cols)
experiments.loc[len(experiments)] = [f"baseline_{int(time.time())}", "tfidf_word","logreg",20000,0,"isotonic",None,None,None,None,None,"init",time.strftime("%Y-%m-%d %H:%M:%S")]
experiments

,run_id,features,model,selectk,svd,calibration,AP,ROC_AUC,F1@0.5,F1@t*,threshold*,notes,created
0,baseline_1757666982,tfidf_word,logreg,20000,0,isotonic,None,None,None,None,None,init,2025-09-12 10:49:42



---
## ✅ Rappel compétence & validation
**Compétence :** *C5.1.2 — Stratégie de résolution*  
**Pourquoi c’est validé (v2) :**
- **Pas de re-EDA / re-normalisation** : on consomme le jeu **normalisé** de C5.1.1 (séparation claire).
- **Stratégie** détaillée (tâches, baselines, extensions HF), **KPI** rappelés.
- **Plan d’expériences** itératif et **registre** prêt à l’usage pour C5.2.x.
